# function to extract data and create a dataframe

In [ ]:
import sys
import os

sys.path.append(os.path.abspath(".."))

import xarray as xr
from config import * 

file_path = os.path.join(NETCDF_FILE, "01012000_rrs_mediterranean_cmems_monthly.nc")
data = xr.open_dataset(file_path, decode_times=True)

csv_path = os.path.join(CSV_DIR, "jan.csv")
df_csv = data.to_dataframe().reset_index()
df_csv.to_csv(csv_path, index=False)

df_rrs = data.to_dataframe().dropna().reset_index()
df_lat_lon = data.to_dataframe().reset_index()

## there are NaNs values and the time is 2002-12-31

              time        lat        lon    RRS412    RRS443    RRS490  \
0       2002-12-31  30.283699  19.112167  0.006137  0.007948  0.008458   
1       2002-12-31  30.283699  19.125019  0.007357  0.009227  0.010133   
2       2002-12-31  30.283699  19.137871  0.004447  0.006845  0.008546   
3       2002-12-31  30.283699  19.150723  0.004044  0.006520  0.008117   
4       2002-12-31  30.283699  19.176426  0.004904  0.007569  0.009212   
...            ...        ...        ...       ...       ...       ...   
1886850 2002-12-31  45.755379  13.624504  0.004613  0.006045  0.008161   
1886851 2002-12-31  45.755379  13.637356  0.004637  0.006245  0.008414   
1886852 2002-12-31  45.765511  13.573097  0.005970  0.006392  0.008068   
1886853 2002-12-31  45.765511  13.585949  0.005043  0.006059  0.008145   
1886854 2002-12-31  45.765511  13.611652  0.004144  0.005768  0.008160   

           RRS510    RRS555    RRS670  
0        0.006160  0.003345  0.000668  
1        0.007656  0.004360  0.